# **Kulik descriptor**

**Author:** Raissa Lohanna

**Problem class:** Regression

**Objective:** Generate Kulik's descriptors for the PubChemQC dataset after EDA and cleaning process

## Importing libraries and configuration

In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
from rdkit import Chem
from tqdm import tqdm
import matplotlib.pyplot as plt

In [35]:
tqdm.pandas()

In [55]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
sns.set_theme(style="darkgrid", palette="husl", rc={"figure.figsize":(10, 5)})

## Importing the dataset

In [4]:
df = pd.read_parquet("joined_data.parquet")

In [5]:
df.head()

,isomeric_smiles,charge,total_dipole_moment,multiplicity,homo,lumo,gap,total_energy,TD_ET_00,TD_OS_00,TD_ET_01,TD_OS_01,TD_ET_02,TD_OS_02,TD_ET_03,TD_OS_03,TD_ET_04,TD_OS_04,TD_ET_05,TD_OS_05,TD_ET_06,TD_OS_06,TD_ET_07,TD_OS_07,TD_ET_08,TD_OS_08,TD_ET_09,TD_OS_09
cid,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,0,12.362788,1,-4.655868,-0.302046,4.353822,-19287.579176,32381.393970,0.002393,33414.775701,0.002229,35041.644378,0.000038,35966.609062,0.004932,37121.320776,0.000746,37641.451596,0.006370,37976.268471,0.001117,38311.230525,0.001396,40062.494092,0.003212,40525.343416,0.002180
2,CC(=O)OC(CC(=O)O)C[N+](C)(C)C,1,5.855433,1,-11.066870,-3.551086,7.515785,-19299.202955,46441.291104,0.000402,46825.670748,0.000412,56096.054110,0.000646,56938.129196,0.022929,57372.999148,0.000343,58598.534406,0.005652,59362.010763,0.005563,60451.940025,0.002308,62136.396688,0.025338,62379.605110,0.017436
3,C1=CC(C(C(=C1)C(=O)O)O)O,0,5.266252,1,-6.821894,-2.239497,4.582397,-15575.874068,32576.410766,0.053438,34937.703709,0.127298,35191.469929,0.035610,38959.724483,0.045520,45869.452080,0.002901,46837.502902,0.004995,48214.775245,0.008408,48851.170886,0.003279,50890.898644,0.046370,51729.941085,0.009966
4,CC(CN)O,0,2.681395,1,-6.187869,1.847653,8.035522,-6794.535860,45133.600212,0.003867,50498.445390,0.022722,52102.569231,0.001101,53565.199229,0.044094,54291.396641,0.024346,58571.208342,0.009868,60147.530252,0.014954,60319.624771,0.008397,60994.089781,0.004093,62458.026397,0.000847
5,C(C(=O)COP(=O)(O)O)N,0,8.447997,1,-7.270882,-1.529280,5.741602,-24256.843704,34507.648887,0.001015,41628.443712,0.008713,45462.755074,0.001559,46943.403498,0.000598,48366.754296,0.003638,50735.499802,0.000874,53071.087855,0.009599,54805.905776,0.025285,54979.815042,0.000488,55740.210361,0.013173


## Applying the Kulik's decriptor

Equação (https://pubs.acs.org/doi/full/10.1021/ja401184g):

$P_d = \sum_i \sum_j P_i P_j \delta(d - d_{ij})$


Importing the elements database

In [7]:
elem_data = pd.read_csv('Atomic properties DB.csv')

In [8]:
elem_data.head()

,Z,Name,Symbol,Average atomic mass,Electronegativity (Pauling),First Ionization Energy (eV),Atomic Radii (pm),Van der Waals Radii (pm),Covalent Radii (pm),Valence electrons
0,1,Hydrogen,H,1007,2.2,1.359844e+06,25,120,38,1.0
1,2,Helium,He,4.002602(2),—,2.458741e+06,31,140,32,2.0
2,3,Lithium,Li,6.941(2),0.98,5.391720e+05,145,182,134,1.0
3,4,Beryllium,Be,9.012182(3),1.57,9.322700e+00,105,—,90,2.0
4,5,Boron,B,10.811(7),02.04,8.298030e+05,85,—,82,3.0


In [9]:
elem_data.dtypes

Z                                 int64
Name                             object
Symbol                           object
Average atomic mass              object
Electronegativity (Pauling)      object
First Ionization Energy (eV)    float64
Atomic Radii (pm)                object
Van der Waals Radii (pm)         object
Covalent Radii (pm)               int64
Valence electrons               float64
dtype: object

Fixing null data and type of objects

In [10]:
elem_data['Electronegativity (Pauling)'] = np.where( 
    ( elem_data['Electronegativity (Pauling)']=='—'), np.NaN, elem_data['Electronegativity (Pauling)'])

elem_data['Atomic Radii (pm)'] = np.where( 
    ( elem_data['Atomic Radii (pm)']=='—'), np.NaN, elem_data['Atomic Radii (pm)'])

elem_data['Van der Waals Radii (pm)'] = np.where( 
    ( elem_data['Van der Waals Radii (pm)']=='—'), np.NaN, elem_data['Van der Waals Radii (pm)'])

elem_data['Valence electrons'] = np.where( 
    ( elem_data['Valence electrons']=='—'), np.NaN, elem_data['Valence electrons'])

In [11]:
float_data = ['Electronegativity (Pauling)', 'Atomic Radii (pm)', 'Van der Waals Radii (pm)']

elem_data[float_data] = elem_data[float_data].astype(float)

In [12]:
elem_data.dtypes

Z                                 int64
Name                             object
Symbol                           object
Average atomic mass              object
Electronegativity (Pauling)     float64
First Ionization Energy (eV)    float64
Atomic Radii (pm)               float64
Van der Waals Radii (pm)        float64
Covalent Radii (pm)               int64
Valence electrons               float64
dtype: object

In [13]:
#Converting the df into a dictionary to be used in the Kulik's descriptor

'''
    Converting DataFrame to Dicionary using a COLUMN as KEY, in the format:
    
    prop: {<ELEMENT_1>: (prop1, prop2, prop3,...),
            <ELEMENT_2>: (prop1, prop2, prop3,...),...
            }
    
    Lengh of list of properties: 10
    
    Index of each property:
    
        0: index
        1: Z
        2: Name
        3: Average atomic mass
        4: Electronegativity (Pauling)
        5: First Ionization Energy (eV)
        6: Atomic Radii (pm)
        7: Van der Waals Radii (pm)
        8: Covalent Radii (pm)
        9: Valence electrons 
'''

properties_dict = elem_data.set_index('Symbol').T.to_dict('list')

#### Applying the Kulik's descriptor

In [61]:
def create_property_descriptors(smiles, depth, prop, prop_index=0):
    '''
    Função:
        create_property_descriptors(smiles, depth, prop, prop_index=0)
    
    Descrição:
        Permite criar descritores baseados em propriedades atômicas e conectividade básica entre os átomos

    Argumentos:
        smiles:     string SMILES da molécula que se deseja gerar os descritores. Exemplo: 'C(C)CO'.
        depth:      profundidade, parâmetro que permite definir o nivel máximo de conectividade que será 
                    usado na geração dos descritores. Exemplo, se depth=3, três descritores serão gerados,
                    com conectividade iguais a um, dois e três.
        prop:       dicionário formado por pares 'átomo': propriedade ou 'átomo':[propriedades] ou 
                    'átomo':(propriedades).
        prop_index: índice da propriedade de interesse nos valores de 'prop'. Se estes valores não forem
                    listas ou tuplas, este parâmetro é ignorado. Se forem, este parâmetro deve ser
                    configurado com um número maior ou igual a zero e menor do que o número de itens nas
                    listas ou tuplas. O valor default deste parâmetro é zero, ou seja, a função pegará a
                    primeira propriedade da lista.
    '''
    try:
        # Criar a estrutura a partir do SMILES da molécula, adicionando os H faltantes
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)
        
        # Preparar listas de resultados (separação por depth=0, 1, 2, 3...)
        result_list = [0] * (depth+1) # Se depth = 2, precisamos de 3 itens (para d=[0,1,2])
            
        # Obter a matriz de distâncias entre cada um dos átomos e a lista de átomos
        atom_list = [ atom.GetSymbol() for atom in mol.GetAtoms() ]
        n_atoms = len(atom_list)
        dist_matrix = np.tril(Chem.rdmolops.GetDistanceMatrix(mol, force=True))
            
        # Iterar toda a matriz, extraindo os dados de interesse e separando por valor de depth
        for i in range(0, n_atoms):
            for j in range(0, n_atoms):
                d = int(dist_matrix[i][j])
                
                if d <= depth and d == 0 and i == j:
                    a_i = atom_list[i]
                    p_i = 0
                    
                    if prop[a_i] is list or tuple:  
                        p_i = prop[a_i][prop_index]   # caso haja + de 1 propriedade p/ calcular
                        
                    else:
                        p_i = prop[a_i]            # caso haja somente 1 propriedade p/ calcular
                        
                    result_list[d] += round(p_i * p_i, 3)
                
                if d <= depth and d > 0:
                    a_i = atom_list[i]
                    a_j = atom_list[j]
                    p_i, p_j = 0, 0
                    
                    if prop[a_i] is list or tuple:  
                        p_i = prop[a_i][prop_index]   # caso haja + de 1 propriedade p/ calcular
                    else:
                        p_i = prop[a_i]            # caso haja somente 1 propriedade p/ calcular
                    if prop[a_j] is list or tuple:
                        p_j = prop[a_j][prop_index]
                    else:
                        p_j = prop[a_j]
                    result_list[d] += round(p_i * p_j, 3)    # posição [d] na lista de resultados é incrementada
        
        # Finalizando
        return result_list

    except TypeError:

        #return smiles
        return print(TypeError)

**Property 3: Electronegativity (Pauling)**

In [87]:
df['Kulik_electronegativity'] = df['isomeric_smiles'].progress_apply(lambda x: create_property_descriptors(smiles=x, depth=3, prop=properties_dict, prop_index=3))

  1%|          | 17396/3332473 [00:43<1:50:22, 500.60it/s]

<class 'TypeError'>


  1%|          | 18389/3332473 [00:45<1:39:12, 556.78it/s]

<class 'TypeError'>


  1%|          | 24226/3332473 [01:01<2:25:52, 377.99it/s]

...

 87%|████████▋ | 2895269/3332473 [1:08:30<09:47, 743.61it/s][22:19:09] WARNING: not removing hydrogen atom without neighbors
[22:19:09] WARNING: not removing hydrogen atom without neighbors
[22:19:09] WARNING: not removing hydrogen atom without neighbors
[22:19:09] WARNING: not removing hydrogen atom without neighbors
 87%|████████▋ | 2904631/3332473 [1:08:42<09:09, 779.14it/s][22:19:21] WARNING: not removing hydrogen atom without neighbors
[22:19:21] WARNING: not removing hydrogen atom without neighbors
 88%|████████▊ | 2920765/3332473 [1:09:03<07:42, 889.54it/s][22:19:42] WARNING: not removing hydrogen atom without neighbors
[22:19:42] WARNING: not removing hydrogen atom without neighbors
[22:19:42] WARNING: not removing hydrogen atom without neighbors
[22:19:42] WARNING: not removing hydrogen atom without neighbors
[22:19:42] WARNING: not removing hydrogen atom without neighbors
[22:19:42] WARNING: not removing hydrogen atom without neighbors
 88%|████████▊ | 2929579/3332473 [1:09:1

<class 'TypeError'>


 96%|█████████▋| 3210948/3332473 [1:15:01<02:04, 973.16it/s][22:25:41] Explicit valence for atom # 3 Cl, 2, is greater than permitted
[22:25:41] WARNING: not removing hydrogen atom without neighbors
[22:25:41] WARNING: not removing hydrogen atom without neighbors
[22:25:41] WARNING: not removing hydrogen atom without neighbors
 96%|█████████▋| 3211089/3332473 [1:15:01<01:54, 1062.77it/s]

<class 'TypeError'>


 98%|█████████▊| 3279291/3332473 [1:16:23<00:55, 961.40it/s] [22:27:02] WARNING: not removing hydrogen atom without neighbors
[22:27:02] WARNING: not removing hydrogen atom without neighbors
 99%|█████████▉| 3294340/3332473 [1:16:40<00:40, 932.41it/s][22:27:19] WARNING: not removing hydrogen atom without neighbors
[22:27:19] WARNING: not removing hydrogen atom without neighbors
100%|█████████▉| 3319240/3332473 [1:17:09<00:16, 821.71it/s] [22:27:49] WARNING: not removing hydrogen atom without neighbors
[22:27:49] WARNING: not removing hydrogen atom without neighbors
[22:27:49] WARNING: not removing hydrogen atom without neighbors
[22:27:49] WARNING: not removing hydrogen atom without neighbors
[22:27:49] WARNING: not removing hydrogen atom without neighbors
100%|██████████| 3332473/3332473 [1:17:26<00:00, 717.27it/s] 


In [88]:
df['Kulik_1st_ionization_energyy'] = df['isomeric_smiles'].progress_apply(lambda x: create_property_descriptors(smiles=x, depth=3, prop=properties_dict, prop_index=4))

  1%|          | 17398/3332473 [00:38<1:36:23, 573.24it/s]

<class 'TypeError'>


  1%|          | 18380/3332473 [00:39<1:16:17, 723.98it/s]

<class 'TypeError'>


  1%|          | 24285/3332473 [00:48<1:16:38, 719.41it/s]

...

 87%|████████▋ | 2895328/3332473 [4:15:33<10:33, 689.99it/s][02:43:39] WARNING: not removing hydrogen atom without neighbors
[02:43:39] WARNING: not removing hydrogen atom without neighbors
[02:43:39] WARNING: not removing hydrogen atom without neighbors
[02:43:39] WARNING: not removing hydrogen atom without neighbors
 87%|████████▋ | 2904622/3332473 [4:15:46<10:45, 663.20it/s][02:43:52] WARNING: not removing hydrogen atom without neighbors
[02:43:52] WARNING: not removing hydrogen atom without neighbors
 88%|████████▊ | 2920745/3332473 [4:16:09<10:31, 651.94it/s][02:44:15] WARNING: not removing hydrogen atom without neighbors
[02:44:15] WARNING: not removing hydrogen atom without neighbors
[02:44:15] WARNING: not removing hydrogen atom without neighbors
[02:44:15] WARNING: not removing hydrogen atom without neighbors
[02:44:15] WARNING: not removing hydrogen atom without neighbors
[02:44:15] WARNING: not removing hydrogen atom without neighbors
 88%|████████▊ | 2935734/3332473 [4:16:3

<class 'TypeError'>


 96%|█████████▋| 3210956/3332473 [4:23:16<02:23, 848.00it/s][02:51:21] Explicit valence for atom # 3 Cl, 2, is greater than permitted
[02:51:21] WARNING: not removing hydrogen atom without neighbors
[02:51:21] WARNING: not removing hydrogen atom without neighbors
 96%|█████████▋| 3211126/3332473 [4:23:16<02:26, 829.18it/s]

<class 'TypeError'>


 98%|█████████▊| 3279318/3332473 [4:24:58<01:04, 828.69it/s] [02:53:04] WARNING: not removing hydrogen atom without neighbors
[02:53:04] WARNING: not removing hydrogen atom without neighbors
 99%|█████████▉| 3294300/3332473 [4:25:20<01:01, 622.14it/s][02:53:26] WARNING: not removing hydrogen atom without neighbors
[02:53:26] WARNING: not removing hydrogen atom without neighbors
[02:53:26] WARNING: not removing hydrogen atom without neighbors
100%|█████████▉| 3319259/3332473 [4:25:58<00:21, 615.54it/s][02:54:03] WARNING: not removing hydrogen atom without neighbors
[02:54:03] WARNING: not removing hydrogen atom without neighbors
[02:54:03] WARNING: not removing hydrogen atom without neighbors
[02:54:03] WARNING: not removing hydrogen atom without neighbors
[02:54:03] WARNING: not removing hydrogen atom without neighbors
100%|██████████| 3332473/3332473 [4:26:17<00:00, 208.58it/s] 


In [89]:
df['Kulik_atomic_radii'] = df['isomeric_smiles'].progress_apply(lambda x: create_property_descriptors(smiles=x, depth=3, prop=properties_dict, prop_index=5))

  1%|          | 17337/3332473 [00:34<1:38:09, 562.85it/s]

<class 'TypeError'>


  1%|          | 18395/3332473 [00:35<1:03:41, 867.15it/s]

<class 'TypeError'>


  1%|          | 24272/3332473 [00:44<1:14:59, 735.22it/s]

...

 87%|████████▋ | 2895320/3332473 [4:01:41<05:42, 1277.32it/s][06:56:04] WARNING: not removing hydrogen atom without neighbors
[06:56:04] WARNING: not removing hydrogen atom without neighbors
[06:56:04] WARNING: not removing hydrogen atom without neighbors
[06:56:04] WARNING: not removing hydrogen atom without neighbors
 87%|████████▋ | 2904613/3332473 [4:01:49<05:52, 1213.16it/s][06:56:12] WARNING: not removing hydrogen atom without neighbors
[06:56:12] WARNING: not removing hydrogen atom without neighbors
 88%|████████▊ | 2920689/3332473 [4:02:02<05:28, 1255.20it/s][06:56:25] WARNING: not removing hydrogen atom without neighbors
[06:56:25] WARNING: not removing hydrogen atom without neighbors
[06:56:25] WARNING: not removing hydrogen atom without neighbors
[06:56:25] WARNING: not removing hydrogen atom without neighbors
[06:56:25] WARNING: not removing hydrogen atom without neighbors
[06:56:25] WARNING: not removing hydrogen atom without neighbors
 88%|████████▊ | 2929501/3332473 [4:0

<class 'TypeError'>


[07:00:20] WARNING: not removing hydrogen atom without neighbors
 96%|█████████▋| 3210924/3332473 [4:05:57<01:24, 1430.82it/s][07:00:20] Explicit valence for atom # 3 Cl, 2, is greater than permitted
[07:00:20] WARNING: not removing hydrogen atom without neighbors
[07:00:20] WARNING: not removing hydrogen atom without neighbors
[07:00:20] WARNING: not removing hydrogen atom without neighbors
 96%|█████████▋| 3211212/3332473 [4:05:57<01:24, 1429.00it/s]

<class 'TypeError'>


 98%|█████████▊| 3279206/3332473 [4:06:53<00:37, 1407.96it/s][07:01:16] WARNING: not removing hydrogen atom without neighbors
[07:01:16] WARNING: not removing hydrogen atom without neighbors
 99%|█████████▉| 3294276/3332473 [4:07:05<00:30, 1255.06it/s][07:01:28] WARNING: not removing hydrogen atom without neighbors
[07:01:28] WARNING: not removing hydrogen atom without neighbors
[07:01:28] WARNING: not removing hydrogen atom without neighbors
100%|█████████▉| 3319150/3332473 [4:07:26<00:11, 1164.83it/s][07:01:49] WARNING: not removing hydrogen atom without neighbors
[07:01:49] WARNING: not removing hydrogen atom without neighbors
[07:01:49] WARNING: not removing hydrogen atom without neighbors
[07:01:49] WARNING: not removing hydrogen atom without neighbors
[07:01:49] WARNING: not removing hydrogen atom without neighbors
100%|██████████| 3332473/3332473 [4:07:36<00:00, 224.31it/s] 


In [90]:
df['Kulik_van_der_waals_radii'] = df['isomeric_smiles'].progress_apply(lambda x: create_property_descriptors(smiles=x, depth=3, prop=properties_dict, prop_index=6))

  1%|          | 17579/3332473 [00:17<48:27, 1140.10it/s]

<class 'TypeError'>


  1%|          | 18439/3332473 [00:18<40:50, 1352.64it/s]

<class 'TypeError'>


  1%|          | 24471/3332473 [00:22<39:20, 1401.24it/s]

...

 87%|████████▋ | 2895264/3332473 [41:28<05:50, 1245.76it/s][07:43:28] WARNING: not removing hydrogen atom without neighbors
[07:43:28] WARNING: not removing hydrogen atom without neighbors
[07:43:28] WARNING: not removing hydrogen atom without neighbors
[07:43:28] WARNING: not removing hydrogen atom without neighbors
 87%|████████▋ | 2904563/3332473 [41:35<05:35, 1276.25it/s][07:43:35] WARNING: not removing hydrogen atom without neighbors
[07:43:35] WARNING: not removing hydrogen atom without neighbors
 88%|████████▊ | 2920746/3332473 [41:48<05:18, 1291.30it/s][07:43:48] WARNING: not removing hydrogen atom without neighbors
[07:43:48] WARNING: not removing hydrogen atom without neighbors
[07:43:48] WARNING: not removing hydrogen atom without neighbors
[07:43:48] WARNING: not removing hydrogen atom without neighbors
[07:43:48] WARNING: not removing hydrogen atom without neighbors
[07:43:48] WARNING: not removing hydrogen atom without neighbors
 88%|████████▊ | 2929564/3332473 [41:55<05:

<class 'TypeError'>


 96%|█████████▋| 3211009/3332473 [3:47:15<01:23, 1446.38it/s][10:49:15] WARNING: not removing hydrogen atom without neighbors
[10:49:15] WARNING: not removing hydrogen atom without neighbors
[10:49:15] WARNING: not removing hydrogen atom without neighbors
 96%|█████████▋| 3211314/3332473 [3:47:15<01:21, 1478.25it/s]

<class 'TypeError'>


[10:49:15] WARNING: not removing hydrogen atom without neighbors
 98%|█████████▊| 3279303/3332473 [3:48:10<00:43, 1222.94it/s][10:50:10] WARNING: not removing hydrogen atom without neighbors
[10:50:10] WARNING: not removing hydrogen atom without neighbors
 99%|█████████▉| 3294292/3332473 [3:48:26<00:28, 1352.53it/s][10:50:26] WARNING: not removing hydrogen atom without neighbors
[10:50:26] WARNING: not removing hydrogen atom without neighbors
[10:50:26] WARNING: not removing hydrogen atom without neighbors
100%|█████████▉| 3319234/3332473 [3:48:46<00:10, 1261.98it/s][10:50:46] WARNING: not removing hydrogen atom without neighbors
[10:50:46] WARNING: not removing hydrogen atom without neighbors
[10:50:46] WARNING: not removing hydrogen atom without neighbors
[10:50:46] WARNING: not removing hydrogen atom without neighbors
[10:50:46] WARNING: not removing hydrogen atom without neighbors
100%|██████████| 3332473/3332473 [3:48:56<00:00, 242.59it/s] 


In [91]:
df['Kulik_covalent_radii'] = df['isomeric_smiles'].progress_apply(lambda x: create_property_descriptors(smiles=x, depth=3, prop=properties_dict, prop_index=7))

  1%|          | 17521/3332473 [00:16<48:21, 1142.35it/s]

<class 'TypeError'>


  1%|          | 18447/3332473 [00:17<37:43, 1464.03it/s]

<class 'TypeError'>


  1%|          | 24344/3332473 [00:21<37:58, 1451.92it/s]

...

 87%|████████▋ | 2895199/3332473 [38:42<05:19, 1369.63it/s][11:29:39] WARNING: not removing hydrogen atom without neighbors
[11:29:39] WARNING: not removing hydrogen atom without neighbors
[11:29:39] WARNING: not removing hydrogen atom without neighbors
[11:29:39] WARNING: not removing hydrogen atom without neighbors
 87%|████████▋ | 2904613/3332473 [38:48<05:19, 1340.53it/s][11:29:46] WARNING: not removing hydrogen atom without neighbors
[11:29:46] WARNING: not removing hydrogen atom without neighbors
 88%|████████▊ | 2920759/3332473 [39:00<04:56, 1390.81it/s][11:29:57] WARNING: not removing hydrogen atom without neighbors
[11:29:57] WARNING: not removing hydrogen atom without neighbors
[11:29:57] WARNING: not removing hydrogen atom without neighbors
[11:29:57] WARNING: not removing hydrogen atom without neighbors
[11:29:57] WARNING: not removing hydrogen atom without neighbors
[11:29:57] WARNING: not removing hydrogen atom without neighbors
 88%|████████▊ | 2929488/3332473 [39:06<04:

<class 'TypeError'>


[11:33:31] WARNING: not removing hydrogen atom without neighbors
 96%|█████████▋| 3210896/3332473 [42:34<01:27, 1389.24it/s][11:33:31] Explicit valence for atom # 3 Cl, 2, is greater than permitted
[11:33:31] WARNING: not removing hydrogen atom without neighbors
[11:33:31] WARNING: not removing hydrogen atom without neighbors
[11:33:31] WARNING: not removing hydrogen atom without neighbors
 96%|█████████▋| 3211241/3332473 [42:34<01:21, 1487.45it/s]

<class 'TypeError'>


 98%|█████████▊| 3279261/3332473 [43:25<00:34, 1531.01it/s][11:34:22] WARNING: not removing hydrogen atom without neighbors
[11:34:22] WARNING: not removing hydrogen atom without neighbors
 99%|█████████▉| 3294220/3332473 [43:35<00:26, 1462.66it/s][11:34:33] WARNING: not removing hydrogen atom without neighbors
[11:34:33] WARNING: not removing hydrogen atom without neighbors
[11:34:33] WARNING: not removing hydrogen atom without neighbors
100%|█████████▉| 3319136/3332473 [43:54<00:10, 1333.64it/s][11:34:51] WARNING: not removing hydrogen atom without neighbors
[11:34:51] WARNING: not removing hydrogen atom without neighbors
[11:34:51] WARNING: not removing hydrogen atom without neighbors
[11:34:51] WARNING: not removing hydrogen atom without neighbors
[11:34:51] WARNING: not removing hydrogen atom without neighbors
100%|██████████| 3332473/3332473 [44:03<00:00, 1260.40it/s]


In [92]:
df['Kulik_valence_e'] = df['isomeric_smiles'].progress_apply(lambda x: create_property_descriptors(smiles=x, depth=3, prop=properties_dict, prop_index=8))

  1%|          | 17530/3332473 [00:17<50:45, 1088.33it/s]

<class 'TypeError'>


  1%|          | 18517/3332473 [00:18<40:36, 1360.13it/s]

<class 'TypeError'>


  1%|          | 24397/3332473 [00:23<40:34, 1358.73it/s]

...

 87%|████████▋ | 2895217/3332473 [3:43:36<05:52, 1241.10it/s][15:18:37] WARNING: not removing hydrogen atom without neighbors
[15:18:37] WARNING: not removing hydrogen atom without neighbors
[15:18:37] WARNING: not removing hydrogen atom without neighbors
[15:18:37] WARNING: not removing hydrogen atom without neighbors
 87%|████████▋ | 2904577/3332473 [3:43:43<05:50, 1219.24it/s][15:18:45] WARNING: not removing hydrogen atom without neighbors
[15:18:45] WARNING: not removing hydrogen atom without neighbors
 88%|████████▊ | 2920711/3332473 [3:43:56<05:21, 1279.51it/s][15:18:57] WARNING: not removing hydrogen atom without neighbors
[15:18:57] WARNING: not removing hydrogen atom without neighbors
[15:18:57] WARNING: not removing hydrogen atom without neighbors
[15:18:57] WARNING: not removing hydrogen atom without neighbors
[15:18:57] WARNING: not removing hydrogen atom without neighbors
[15:18:57] WARNING: not removing hydrogen atom without neighbors
 88%|████████▊ | 2929522/3332473 [3:4

<class 'TypeError'>


 96%|█████████▋| 3210928/3332473 [3:47:50<01:25, 1420.07it/s][15:22:52] Explicit valence for atom # 3 Cl, 2, is greater than permitted
[15:22:52] WARNING: not removing hydrogen atom without neighbors
[15:22:52] WARNING: not removing hydrogen atom without neighbors
[15:22:52] WARNING: not removing hydrogen atom without neighbors
 96%|█████████▋| 3211239/3332473 [3:47:50<01:23, 1452.02it/s]

<class 'TypeError'>


 98%|█████████▊| 3279208/3332473 [3:48:47<00:36, 1452.87it/s][15:23:49] WARNING: not removing hydrogen atom without neighbors
[15:23:49] WARNING: not removing hydrogen atom without neighbors
 99%|█████████▉| 3294340/3332473 [3:48:59<00:28, 1349.77it/s][15:24:01] WARNING: not removing hydrogen atom without neighbors
[15:24:01] WARNING: not removing hydrogen atom without neighbors
100%|█████████▉| 3319231/3332473 [3:49:19<00:10, 1227.47it/s][15:24:21] WARNING: not removing hydrogen atom without neighbors
[15:24:21] WARNING: not removing hydrogen atom without neighbors
[15:24:21] WARNING: not removing hydrogen atom without neighbors
[15:24:21] WARNING: not removing hydrogen atom without neighbors
[15:24:21] WARNING: not removing hydrogen atom without neighbors
100%|██████████| 3332473/3332473 [3:49:30<00:00, 242.00it/s] 


Saving data:

In [93]:
df.to_parquet("data_outputs_w_kulik.parquet")